In [1]:

import tensorflow as tf 


In [2]:
from tensorflow.keras.applications import InceptionV3 
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Dropout, Input, Flatten,Dense,MaxPooling2D 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 


In [9]:
tf.test.is_gpu_available()

False

#### Performing Data Agumentation in Training Data by:-
<ol>
<li>Rotaion</li>
<li>Shearing </li>
<li>Zooming</li>
<li>Shifting width </li>
</ol>
<hr/>
Rescaling the training data by dividing it to 255 so as to have a normalized training data set
and Dividing the training data to validation data by 20%

In [10]:
train_data_gen=ImageDataGenerator(rotation_range=0.2,shear_range=0.2,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,rescale=1/255,validation_split=0.2)

### Training data

In [11]:
train_data = train_data_gen.flow_from_directory(r'C:\Users\Hp\Desktop\mrlEyes_2018_01\Prepared_Data\Train',target_size=(80,80), batch_size = 8, class_mode='categorical',subset='training')

Found 61146 images belonging to 2 classes.


### Validation Data

In [12]:
validation_data = train_data_gen.flow_from_directory(r'C:\Users\Hp\Desktop\mrlEyes_2018_01\Prepared_Data\Train',target_size=(80,80), batch_size = 8, class_mode='categorical',subset='validation')

Found 15286 images belonging to 2 classes.


### Test Data

In [13]:
# Dividing by 255 to normalize the pixxel size
test_data_gen=ImageDataGenerator(rescale=1/255)

In [14]:
test_data = test_data_gen.flow_from_directory(r'C:\Users\Hp\Desktop\mrlEyes_2018_01\Prepared_Data\Test',target_size=(80,80), batch_size = 8, class_mode='categorical')


Found 8466 images belonging to 2 classes.


#### Performing Transfer Learning in InceptionV3 CNN Architecture
<ol>
<li>Flatten its end model layers and overwrite by our own model layers </li>
<li>Made its base model layers non-traininable </li>
<li>Dropout regularization method is used to reduce overfitting</li>
</ol>
 Example of flattening :-
<img src="https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/73_blog_image_1.png">


Example of Transfer Learning :-

<img class="lazyload" src="https://www.analyticssteps.com/backend/media/thumbnail/1967565/9315476_1592890541_transfer.jpg" alt="How is Transfer Learning done in Neural Networks and Convolutional Neural Networks? title banner" title="How is Transfer Learning done in Neural Networks and Convolutional Neural Networks?">

In [15]:
bmodel = InceptionV3(include_top=False,weights='imagenet',input_tensor=Input(shape=(80,80,3)))
hmodel=bmodel.output
hmodel=Flatten()(hmodel)
hmodel=Dense(64,activation='relu')(hmodel)
hmodel=Dropout(0.5)(hmodel)
hmodel = Dense(2,activation='softmax')(hmodel)

##Model all base layers non trainable 
model = Model(inputs=bmodel.input,outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable=False

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

### Saving the models to a specified directory --

<ol>
<li>Model Checkpoint will save the model that have least validation loss</li>
<li>In each saving of model, their weights will replace by best updated weights</li>
<li>Early stopping is enabled so model will stop training if its not see any further decrease or any inscrease in accuracy in validation loss after 7 epochs </li>
<li>If validation loss will not decrease after 2 3 epochs then we will reduce the learning rate</li>
</ol>


In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint=ModelCheckpoint(r'C:\Users\Hp\Desktop\mrlEyes_2018_01\models\model.h5',monitor='val_loss',save_best_only=True,verbose=3)
earlystop=EarlyStopping(monitor='val_loss',restore_best_weights=True,patience=7,verbose=3)

learning_rate=ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3)
callbackss=[checkpoint,earlystop,learning_rate]

### Compiling the model and train the model

In [19]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_data,
                    steps_per_epoch=train_data.samples//8,
                    validation_data=validation_data,
                    validation_steps=validation_data.samples//8,
                    callbacks=callbackss,
                   epochs=5)
# model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14708\615738299.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,


Epoch 1/5
7643/7643 [==============================] - ETA: 0s - loss: 0.1872 - accuracy: 0.9297
Epoch 1: val_loss improved from inf to 0.30961, saving model to C:\Users\Hp\Desktop\mrlEyes_2018_01\models\model.h5
7643/7643 [==============================] - 843s 109ms/step - loss: 0.1872 - accuracy: 0.9297 - val_loss: 0.3096 - val_accuracy: 0.8904 - lr: 0.0010
Epoch 2/5
7643/7643 [==============================] - ETA: 0s - loss: 0.1602 - accuracy: 0.9376
Epoch 2: val_loss improved from 0.30961 to 0.24926, saving model to C:\Users\Hp\Desktop\mrlEyes_2018_01\models\model.h5
7643/7643 [==============================] - 682s 89ms/step - loss: 0.1602 - accuracy: 0.9376 - val_loss: 0.2493 - val_accuracy: 0.9103 - lr: 0.0010
Epoch 3/5
7643/7643 [==============================] - ETA: 0s - loss: 0.1533 - accuracy: 0.9412
Epoch 3: val_loss did not improve from 0.24926
7643/7643 [==============================] - 654s 86ms/step - loss: 0.1533 - accuracy: 0.9412 - val_loss: 0.2574 - val_accuracy

In [20]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14708\713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.1256791204214096
0.9531122446060181


In [21]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14708\4081756742.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.25550025701522827
0.9101792573928833


In [22]:
acc_vr, loss_vr = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14708\2429375605.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(test_data)


0.1256791204214096
0.9531122446060181
